# Housing Prices Machine Learning Project

This notebook documents the workflow for predicting housing prices using machine learning techniques.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load dataset from CSV file
train_data_fp = r'C:\Users\thebi\house-prices-ml-project\data\train.csv'
test_data_fp = r'C:\Users\thebi\house-prices-ml-project\data\test.csv'
df_train = pd.read_csv(train_data_fp)
df_test = pd.read_csv(test_data_fp)

print('Number of Training Examples:', df_train.shape[0])
print('Number of Testing Examples:', df_test.shape[0])
print('Training X Shape:', df_train.shape)
print('Testing X Shape:', df_test.shape)

In [ ]:
# ***EXPLORATORY DATA ANALYSIS***


# Display basic statistics and first few rows of the dataframe
print(df_train.describe())
print(df_train.head())
# print(df_train.columns)
print(df_train.info())
df_train.sample(5)

In [ ]:
# Check for missing values in the dataset
def display_missing(df_train):
    for col in df_train.columns.tolist():
        print('{} column missing values: {}'.format(col, df_train[col].isnull().sum()))
    print('\n')

display_missing(df_train)